<a href="https://colab.research.google.com/github/dev0419/BDA_Lab/blob/main/Lab-6/DecisionTree_L6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

1. Demonstrate how to load a dataset suitable for prediction into a PySpark DataFrame and Display basic statistics and information about the dataset

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('decision_tree').getOrCreate()
df = spark.read.csv('covtype.data',inferSchema=True,header=False)
df.show(5)
colnames = ["Elevation","Aspect","Slope","Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology",\
           "Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_noon","Hillshade_3pm",\
           "Horizontal_Distance_To_Fire_Points"] + \
           [f"Wilderness_Area_{i}" for i in range(4)] + [f"Soil_Type_{i}" for i in range(40)] + ["Cover_Type"]
df = df.toDF(*colnames)
df.show(5)
print("Information about the dataset")
df.printSchema()
print("Summary statistics")
df.describe().show()


+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| _c0|_c1|_c2|_c3|_c4| _c5|_c6|_c7|_c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|_c47|_c48|_c49|_c50|_c51|_c52|_c53|_c54|
+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|2596| 51|  3|258|  0| 510|221|232|148|6279|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0| 

2. Implement a PySpark script to handle missing values and categorical features in the dataset.

In [3]:
from pyspark.sql.functions import col
from pyspark.ml.feature import Imputer

null_counts = [df.where(col(c).isNull()).count() for c in df.columns]
for column, count in zip(df.columns, null_counts):
    print(f"Column '{column}' has {count} null values")

num_features = [
    'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]
cat_features = [
    'Wilderness_Area_0','Wilderness_Area_1','Wilderness_Area_2','Wilderness_Area_3','Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6',
    'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13',
    'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20',
    'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27',
    'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34',
    'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39', 'Cover_Type'
]

num_imputer = Imputer(inputCols=num_features,outputCols=[f"{col}_imputed" for col in num_features])
num_imputed = num_imputer.fit(df)
df_num_imputed =  num_imputed.transform(df)
print("Numerical features imputed")
df_num_imputed.show(5)
cat_imputer = Imputer(inputCols=cat_features,outputCols=[f"{col}_imputed" for col in cat_features])
cat_imputed = cat_imputer.fit(df_num_imputed)
df_imputed = cat_imputed.transform(df_num_imputed)
print("Categorical features imputed")
df_imputed.show(5)


Column 'Elevation' has 0 null values
Column 'Aspect' has 0 null values
Column 'Slope' has 0 null values
Column 'Horizontal_Distance_To_Hydrology' has 0 null values
Column 'Vertical_Distance_To_Hydrology' has 0 null values
Column 'Horizontal_Distance_To_Roadways' has 0 null values
Column 'Hillshade_9am' has 0 null values
Column 'Hillshade_noon' has 0 null values
Column 'Hillshade_3pm' has 0 null values
Column 'Horizontal_Distance_To_Fire_Points' has 0 null values
Column 'Wilderness_Area_0' has 0 null values
Column 'Wilderness_Area_1' has 0 null values
Column 'Wilderness_Area_2' has 0 null values
Column 'Wilderness_Area_3' has 0 null values
Column 'Soil_Type_0' has 0 null values
Column 'Soil_Type_1' has 0 null values
Column 'Soil_Type_2' has 0 null values
Column 'Soil_Type_3' has 0 null values
Column 'Soil_Type_4' has 0 null values
Column 'Soil_Type_5' has 0 null values
Column 'Soil_Type_6' has 0 null values
Column 'Soil_Type_7' has 0 null values
Column 'Soil_Type_8' has 0 null values
Co

3. Develop a PySpark script that trains a decision tree model on the training
dataset.

In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

train,test = df.randomSplit([0.9,0.1])
assembler = VectorAssembler(inputCols=num_features + cat_features,outputCol='features')
train = assembler.transform(train)
train.show(5)
test = assembler.transform(test)
dt = DecisionTreeClassifier(labelCol = 'Cover_Type',featuresCol='features')
dt_model = dt.fit(train)
print(dt_model.toDebugString)
preds = dt_model.transform(test)
preds.show(5)

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+----------+--------------------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_noon|Hillshade_3pm|Horizontal_Distance_To_F

In [9]:
import pandas as pd
pd.DataFrame(dt_model.featureImportances.toArray(),index=df.columns,columns=['importance']).sort_values(by="importance",ascending=False)


,importance
Cover_Type,1.0
Soil_Type_27,0.0
Soil_Type_16,0.0
Soil_Type_17,0.0
Soil_Type_18,0.0
Soil_Type_19,0.0
Soil_Type_20,0.0
Soil_Type_21,0.0
Soil_Type_22,0.0
Soil_Type_23,0.0


4. Implement code to evaluate the decision tree model using metrics such as accuracy, precision, and recall.

In [14]:
evaluator=MulticlassClassificationEvaluator(labelCol="Cover_Type",predictionCol="prediction")
evaluator.setMetricName("accuracy").evaluate(preds)
evaluator.setMetricName("f1").evaluate(preds)

0.9932117776191136